In [14]:
import pandas as pd
from google.cloud import bigquery
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Load data from BigQuery
client = bigquery.Client()
query = """
SELECT * FROM `superb-reporter-430115-t3.sale_output.super_store_sales`
"""
df = client.query(query).to_dataframe()

# Feature engineering
X = df.drop('Sales', axis=1)
y = df['Sales']
# display(X)
# display(y)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train an XGBoost model
model = XGBRegressor(n_estimators=100)
display(X_train)
display(y_train)

model.fit(X_train._get_numeric_data(), y_train._get_numeric_data())

# # Evaluate the model
predictions = model.predict(X_test._get_numeric_data())
print(f'Mean Squared Error: {mean_squared_error(y_test, predictions)}')


,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub-Category,Quantity,Discount,Profit
4100,Second Class,Consumer,United States,Las Cruces,New Mexico,88001,West,Office Supplies,Paper,4,0.0,23.0864
2586,Standard Class,Consumer,United States,Rome,New York,13440,East,Office Supplies,Art,9,0.0,8.3538
9194,Standard Class,Consumer,United States,Johnson City,Tennessee,37604,South,Office Supplies,Paper,7,0.2,12.7008
2002,Same Day,Corporate,United States,Riverside,California,92503,West,Office Supplies,Paper,2,0.0,6.2208
7225,Standard Class,Corporate,United States,San Diego,California,92024,West,Technology,Phones,2,0.2,27.1032
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,Standard Class,Consumer,United States,New York City,New York,10009,East,Furniture,Chairs,7,0.1,65.2064
5191,Same Day,Consumer,United States,Aurora,Illinois,60505,Central,Furniture,Furnishings,1,0.6,-66.0620
5390,Standard Class,Home Office,United States,Tampa,Florida,33614,South,Office Supplies,Binders,2,0.7,-3.5520
860,Standard Class,Corporate,United States,New York City,New York,10024,East,Office Supplies,Art,5,0.0,6.6150


4100     49.120
2586     32.130
9194     36.288
2002     12.960
7225    361.376
         ...   
5734    366.786
5191     77.720
5390      5.328
860      14.700
7270    115.296
Name: Sales, Length: 7995, dtype: float64

Mean Squared Error: 130067.31425986406


In [15]:
import joblib
from google.cloud import storage

# Save the model
joblib.dump(model, 'model.joblib')

# Upload to Cloud Storage
storage_client = storage.Client()
bucket = storage_client.bucket('superb-reporter-430115-t3-ai-retail-model')
blob = bucket.blob('model.joblib')
blob.upload_from_filename('model.joblib')


In [16]:
FROM python:3.8-slim

RUN pip install --no-cache-dir flask joblib google-cloud-storage xgboost

COPY model.joblib /app/model.joblib
COPY app.py /app/app.py

WORKDIR /app

CMD ["python", "app.py"]


SyntaxError: invalid syntax (<ipython-input-16-5219efd4cb13>, line 1)